# Step 5: Predicting engagement based on sentiments (NO LONGER ACTIVE)

This machine learning prediction is no longer in use. I had to scale back the scope of my research after my research partner quit the course at the last minute. 

In [116]:
import pandas as pd
import numpy as np 
from pandas import Series
import seaborn as sns

In [15]:
mlr_titles = pd.read_csv('mlr_titles.csv').drop(['Unnamed: 0'], axis=1)
mlr_titles

,title,score,num_comments,sentiment
0,Dyker Heights murder suspect allegedly killed ...,112,22,-0.9652
1,"Asian man stabbed in back in Chinatown, suspec...",17577,1061,-0.9571
2,'Satanic' neo-Nazi terrorist arrested at Luton...,281,108,-0.9451
3,"Man hires two to kill rape victim, pair kills ...",7735,723,-0.9432
4,Mom helped teen murder suspects flee state; 5 ...,167,17,-0.9393
...,...,...,...,...
6055,West Virginia police giving out gift cards rat...,258,101,0.8834
6056,"Friends share Powerball jackpot win, keeping 1...",1140,82,0.8860
6057,Trump to award trio of golfers Medal of Freedom,1152,269,0.8957
6058,Two French Navy FREMM Frigates Won a U.S. Navy...,99,5,0.9062


In [152]:
#Step 1 
# Define X and y
X =  mlr_titles['sentiment'].values
#X = mlr_titles['score'].values + mlr_titles['sentiment'].values 
y = mlr_titles['score'].values
#Now we split the data into development and test datasets
from sklearn.model_selection import train_test_split
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=161193)
#split development dataset into a training and validation dataset. For 10-fold cross-validation we have a test-size of 0.1
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=161193)

In [153]:
#Step 2
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as mse

In [166]:
pipe = make_pipeline(PolynomialFeatures(degree = 1), StandardScaler(), LinearRegression())

pipe.fit(X_train.reshape(-1,1), y_train.reshape(-1,1)) # have to reshape so that we get 2D array
y_pred = pipe.predict(X_test.reshape(-1,1))

In [167]:
# prep
perform = [] #store performance
lambdas = np.logspace(-4, -4., 20) #grid of lambdas (hyperparameters)

#for each lambda, fit model on training data, check performance on validation and store mse
for lambda_ in lambdas:
    
    pipe_lasso =  make_pipeline(PolynomialFeatures(), StandardScaler(), Lasso(alpha=lambda_, random_state=12465))
    pipe_lasso.fit(X_train.reshape(-1, 1), y_train.reshape(-1, 1))
    y_pred = pipe_lasso.predict(X_val.reshape(-1, 1))
    perform.append(mse(y_pred, y_val.reshape(-1, 1)))

#create series with performance and find parameter with lowest MSE
hyperparam_perform = pd.Series(perform, index=lambdas)
optimal = hyperparam_perform.nsmallest(1)
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.3f' % optimal.values[0])

Optimal lambda: 0.0001
Validation MSE: 231389531.773


In [168]:
# prep
perform = [] #store performance
lambdas = np.logspace(-2, -2., 50) #grid of initial lambdas (hyperparameters)

#for each lambda, fit model on training data, check performance on validation and store mse
for lambda_ in lambdas:
    
    pipe_lasso =  make_pipeline(PolynomialFeatures(), StandardScaler(), Lasso(alpha=lambda_, random_state=12465))
    pipe_lasso.fit(X_train.reshape(-1, 1), y_train.reshape(-1, 1))
    y_pred = pipe_lasso.predict(X_val.reshape(-1, 1))
    perform.append(mse(y_pred, y_val.reshape(-1, 1)))

#create series with performance and find parameter with lowest MSE
hyperparam_perform = pd.Series(perform, index=lambdas)
optimal = hyperparam_perform.nsmallest(1)
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.3f' % optimal.values[0])

Optimal lambda: 0.01
Validation MSE: 231389581.334


In [169]:
#Insert optimal lambda into new model:
pipe_lasso = make_pipeline(PolynomialFeatures(), StandardScaler(), Lasso(alpha=optimal.index[0]))
#Fit new model on all the development data to build best model
pipe_lasso.fit(X_dev.reshape(-1, 1), y_dev.reshape(-1, 1))

#give model performance
print('Lasso', (mse(pipe_lasso.predict(X_test.reshape(-1, 1)),y_test.reshape(-1, 1)),2))

Lasso (246011119.44931573, 2)
